In [85]:
%env JAX_PLATFORM_NAME=cpu

import jaxtyping  # noqa: F401

%load_ext jaxtyping
# %jaxtyping.typechecker beartype.beartype

env: JAX_PLATFORM_NAME=cpu
The jaxtyping extension is already loaded. To reload it, use:
  %reload_ext jaxtyping


In [86]:
import jax
import jax.numpy as jnp
import equinox as eqx
import optax

from tqdm.notebook import trange

from jaxtyping import Array, Bool, Float

from chaogatenn.chaogate import ChaoGate
from chaogatenn.maps import (
    LogisticMap,
    DuffingMap,
    LorenzMap,
    RosslerMap,
    ChenMap,
    RosslerHyperchaosMap,
)
from chaogatenn.utils import grad_norm

In [87]:
# Training data for the AND gate
X = jnp.array([[0, 0], [0, 1], [1, 0], [1, 1]], dtype=bool)  # Input combinations
AND_Y = jnp.array([0, 0, 0, 1], dtype=bool)  # AND gate output
OR_Y = jnp.array([0, 1, 1, 1], dtype=bool)  # OR gate output
XOR_Y = jnp.array([0, 1, 1, 0], dtype=bool)  # XOR
NAND_Y = jnp.array([1, 1, 1, 0], dtype=bool)  # NAND
NOR_Y = jnp.array([1, 0, 0, 0], dtype=bool)  # NOR
XNOR_Y = jnp.array([1, 0, 0, 1], dtype=bool)  # XNOR
Y = NAND_Y

In [88]:
Map = LogisticMap(a=4.0)
# Map = LorenzMap()
# Map = DuffingMap(
#     alpha=1.0, beta=1.0, delta=0.02, gamma=8.0, omega=0.5, dt=0.01, steps=1000
# )
# Map = ChenMap(dt=1e-5, steps=100)
# Map = RosslerHyperchaosMap()

In [89]:
Map(2)  # type: ignore

-8.0

In [90]:
DELTA, X0, X_THRESHOLD = jax.random.normal(jax.random.PRNGKey(42), (3,))
chao_gate = ChaoGate(DELTA=DELTA, X0=X0, X_THRESHOLD=X_THRESHOLD, Map=Map)

In [91]:
[chao_gate(x) for x in X]

[Array(4.0121733e-05, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.0005, dtype=float32),
 Array(0.00469383, dtype=float32)]

In [92]:
@eqx.filter_value_and_grad()
def compute_loss(
    chao_gate: ChaoGate, x: Bool[Array, "batch 2"], y: Bool[Array, "batch"]
) -> Float[Array, ""]:  # noqa: F821
    pred = jax.vmap(chao_gate)(x)
    # binary cross entropy
    return (
        20
        * (-jnp.mean(y * jnp.log(pred + 1e-15) + (1 - y) * jnp.log(1 - pred + 1e-15)))
        + 10 * chao_gate.DELTA**2
        - 5 * (chao_gate.X0 + chao_gate.X_THRESHOLD + chao_gate.DELTA)
    )

In [93]:
@eqx.filter_jit
def make_step(
    model: ChaoGate,
    x: Bool[Array, "dim 2"],
    y: Bool[Array, "dim"],  # noqa: F821
    optim: optax.GradientTransformation,
    opt_state: optax.OptState,
) -> (Float[Array, "dim"], ChaoGate, optax.OptState):  # type: ignore  # noqa: F821
    loss, grads = compute_loss(model, x, y)
    updates, opt_state = optim.update(grads, opt_state)
    # jax.debug.print(f"{grads, updates}")
    model = eqx.apply_updates(model, updates)
    return loss, model, opt_state

In [94]:
optim = optax.adabelief(3e-5)
opt_state = optim.init(eqx.filter(chao_gate, eqx.is_inexact_array))

In [95]:
epochs = 5_000

for epoch in trange(epochs):
    loss, chao_gate, opt_state = make_step(chao_gate, X, Y, optim, opt_state)  # type: ignore
    _, grads = compute_loss(chao_gate, X, Y)
    grad_norm_value = grad_norm(grads)
    if epoch % 10 == 0:
        print(f"Epoch {epoch}, Loss: {loss}, Grad Norm: {grad_norm_value}")

    if loss < 1e-3:
        break

  0%|          | 0/5000 [00:00<?, ?it/s]

Epoch 0, Loss: 140.2650604248047, Grad Norm: 240.60440063476562
Epoch 10, Loss: 140.1385040283203, Grad Norm: 240.4947967529297
Epoch 20, Loss: 139.9593048095703, Grad Norm: 240.341552734375
Epoch 30, Loss: 139.73304748535156, Grad Norm: 240.14926147460938
Epoch 40, Loss: 139.46629333496094, Grad Norm: 239.92323303222656
Epoch 50, Loss: 139.16407775878906, Grad Norm: 239.66720581054688
Epoch 60, Loss: 138.8301239013672, Grad Norm: 239.384033203125
Epoch 70, Loss: 138.46722412109375, Grad Norm: 239.07566833496094
Epoch 80, Loss: 138.07760620117188, Grad Norm: 238.74380493164062
Epoch 90, Loss: 137.6631317138672, Grad Norm: 238.3895721435547
Epoch 100, Loss: 137.22525024414062, Grad Norm: 238.01400756835938
Epoch 110, Loss: 136.7653350830078, Grad Norm: 237.6177978515625
Epoch 120, Loss: 136.2845001220703, Grad Norm: 237.20169067382812
Epoch 130, Loss: 135.78378295898438, Grad Norm: 236.76625061035156
Epoch 140, Loss: 135.2640838623047, Grad Norm: 236.31185913085938
Epoch 150, Loss: 134.

In [96]:
print("\nTrained ChaoGate Parameters:")
print(
    f"DELTA: {chao_gate.DELTA}, X0: {chao_gate.X0}, X_THRESHOLD: {chao_gate.X_THRESHOLD}"
)


Trained ChaoGate Parameters:
DELTA: 0.5360193252563477, X0: 0.21202777326107025, X_THRESHOLD: 0.1779542863368988


In [97]:
[
    (
        bool(x1.item()),
        bool(x2.item()),
        (
            chao_gate.Map(chao_gate.X0 + x1 * chao_gate.DELTA + x2 * chao_gate.DELTA)
            > chao_gate.X_THRESHOLD
        ).item(),
    )
    for x1, x2 in X
]

[(False, False, True),
 (False, True, True),
 (True, False, True),
 (True, True, False)]

In [98]:
pred_ys = jax.vmap(chao_gate)(X)
num_correct = jnp.sum((pred_ys > 0.5) == Y)
final_accuracy = (num_correct / len(X)).item()
print(f"final_accuracy={final_accuracy}")

final_accuracy=1.0
